# Minibatch FFNN Implementation Evaluation
### Group Members 
1. 13516044 - Annisa Sekar Ayuningtyas
2. 13516078 - Muhammad Alif Arifin
3. 13516152 - Deborah Aprilia Josephine

## Import Library and Dataset

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from MiniBatch import MiniBatch

df = pd.read_csv('dataset.csv')
df

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [12]:
df['Outlook'] = df['Outlook'].astype('category')
df['Windy'] = df['Windy'].astype('category')
df['Play'] = df['Play'].astype('category')

df['Outlook'] = df['Outlook'].cat.codes
df['Windy'] = df['Windy'].cat.codes
df['Play'] = df['Play'].cat.codes

df

,Outlook,Temperature,Humidity,Windy,Play
0,2,85,85,0,0
1,2,80,90,1,0
2,0,83,86,0,1
3,1,70,96,0,1
4,1,68,80,0,1
5,1,65,70,1,0
6,0,64,65,1,1
7,2,72,95,0,0
8,2,69,70,0,1
9,1,75,80,0,1


In [13]:
features = df[['Outlook', 'Temperature', 'Humidity', 'Windy']]
targets = df['Play']

print (features.head)
print ()
print (targets.head)

<bound method NDFrame.head of     Outlook  Temperature  Humidity  Windy
0         2           85        85      0
1         2           80        90      1
2         0           83        86      0
3         1           70        96      0
4         1           68        80      0
5         1           65        70      1
6         0           64        65      1
7         2           72        95      0
8         2           69        70      0
9         1           75        80      0
10        2           75        70      1
11        0           72        90      1
12        0           81        75      0
13        1           71        91      1>

<bound method NDFrame.head of 0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Play, dtype: int8>


## Training and Testing

In [15]:
features_train, features_test, targets_train, targets_test = train_test_split(features, targets, test_size=0.5, stratify=targets, random_state=None)
print (features_train)
print ()
print (targets_train)
print ()

print (features_test)
print ()
print (targets_test)
print ()

    Outlook  Temperature  Humidity  Windy
11        0           72        90      1
2         0           83        86      0
6         0           64        65      1
1         2           80        90      1
8         2           69        70      0
7         2           72        95      0
4         1           68        80      0

11    1
2     1
6     1
1     0
8     1
7     0
4     1
Name: Play, dtype: int8

    Outlook  Temperature  Humidity  Windy
0         2           85        85      0
5         1           65        70      1
10        2           75        70      1
12        0           81        75      0
9         1           75        80      0
13        1           71        91      1
3         1           70        96      0

0     0
5     0
10    1
12    1
9     1
13    0
3     1
Name: Play, dtype: int8



In [ ]:
model = MiniBatch(2, 3, 2, 0.1, 0.5, 4)
model.fit(features_train, trgets_train)
predict = model.predict(features_test)

print ('Accuracy\t', accuracy_score(targets_test, predict))